In [33]:
import pandas as pd

import re
import numpy as np
import pandas as pd
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation,  Dropout, Dense, Embedding, Bidirectional

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import keras.utils as ku 

from keras.callbacks import ModelCheckpoint

import pickle

import random as rd

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/glep93/X-mas_song_generator/main/xmas-songs.csv')

In [10]:
def preprocessing(x):
    x = x.lower()
    x = re.sub('\[(.*?)\]', '', x)
    x = re.sub('[\n]{2,}', '\n',x)
    x = re.sub("[^a-z\'\s\n]",'',x)
    x = re.sub('[\n]', ' *newline* ',x)
    x = re.sub('^\s','',x)
    x = re.sub('[\s]{2,}', ' ',x)
    out =np.array( x.split(' ') )
    out = out[out!= '']
    return out 

In [27]:
lyrics =df['lyrics'].dropna()[ df.Artist=='Michael Bublé'][[13,144 ,170 ,289 ,387 ,592]]
lyrics = lyrics.apply(preprocessing )

In [28]:
aux = []
for i in lyrics:
    aux  = np.append(aux, i)

vocab = set( list(aux) )

index_to_word = { i+1: word for i, word in enumerate(vocab) }
word_to_index = {  word: i+1 for i, word in enumerate(vocab) }
index_to_word[0] = 'missing_word'
word_to_index['missing_word'] = 0

total_words = len( vocab)+1
print(total_words)

1387


In [29]:
pickle.dump( index_to_word, open( "index_to_word.p", "wb" ) )
pickle.dump( word_to_index, open( "word_to_index.p", "wb" ) )

In [31]:
n = 30
X = []
y = []

for song in lyrics:
    N = len(song)
    for i in range( N -n- 1):
        for j in range(1,n):
            X.append(  [word_to_index[word]  for word in song[i:i+j]] )
            y.append( word_to_index[song[i+j]])


In [46]:
y = np.array(y)

In [32]:
X_pad = pad_sequences(X, maxlen = n, padding='pre')
X_pad.shape

(133429, 30)

In [44]:
num_sample_training = 100000
tot=len(X_pad)
index_training=np.random.randint(0,tot,num_sample_training)

In [47]:
X_train = X_pad[index_training]
y_train = y[index_training]

In [48]:
model = Sequential()

# Add Input Embedding Layer
model.add(Embedding(total_words, 50, input_length=30))
    
 # Add Hidden Layer 1 - LSTM Layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))
#model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(LSTM(128))

model.add(Dense(total_words/2, activation='relu')) 

# Add Output Layer
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 50)            69350     
_________________________________________________________________
bidirectional (Bidirectional (None, 30, 256)           183296    
_________________________________________________________________
dropout (Dropout)            (None, 30, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense (Dense)                (None, 693)               89397     
_________________________________________________________________
dense_1 (Dense)              (None, 1387)              962578    
Total params: 1,501,741
Trainable params: 1,501,741
Non-trainable params: 0
______________________________________________

In [49]:
label = ku.to_categorical(y, num_classes=total_words)

In [52]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
history = model.fit(X_pad, label, epochs=10, batch_size=512, callbacks=callbacks_list)

Epoch 1/10
261/261 [==============================] - ETA: 0s - loss: 2.1662
Epoch 00001: loss improved from inf to 2.16622, saving model to weights-improvement-01-2.1662.hdf5
261/261 [==============================] - 124s 477ms/step - loss: 2.1662
Epoch 2/10
261/261 [==============================] - ETA: 0s - loss: 1.3819
Epoch 00002: loss improved from 2.16622 to 1.38188, saving model to weights-improvement-02-1.3819.hdf5
261/261 [==============================] - 151s 578ms/step - loss: 1.3819
Epoch 3/10
261/261 [==============================] - ETA: 0s - loss: 0.9261
Epoch 00003: loss improved from 1.38188 to 0.92605, saving model to weights-improvement-03-0.9261.hdf5
261/261 [==============================] - 140s 535ms/step - loss: 0.9261
Epoch 4/10
261/261 [==============================] - ETA: 0s - loss: 0.5943
Epoch 00004: loss improved from 0.92605 to 0.59434, saving model to weights-improvement-04-0.5943.hdf5
261/261 [==============================] - 148s 568ms/step - l

In [53]:
model.save('mAIchael_buble')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: mAIchael_buble/assets


In [56]:
from tensorflow import keras
model_load = keras.models.load_model('mAIchael_buble')

In [57]:
def make_lyrics( input, n_words):
    input_p = preprocessing(input)
    for i in range(n_words):
        input_pad = pad_sequences( [[word_to_index[word] if word in word_to_index.keys() else 0  for word in input_p ]] , maxlen = n, padding='pre')
        predict = model_load.predict_classes(input_pad, verbose=0)[0]
        if index_to_word[predict] == '*newline*':
            input = input + '\n'
        else:
            input = input +' '+ index_to_word[predict] 
        input_p = np.append(input_p, index_to_word[predict])
    return input

In [58]:
input  = '''all i want for christmas'''

print(make_lyrics( input, 200))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
all i want for christmas
 is you
 i won't ask for much this christmas
 i won't even wish for snow
 no i'm just gonna keep on waiting
 underneath the mistletoe
 there's no sense in hanging stockings
 there upon the fireplace
 'cause santa he won't make me happy
 with a toy on christmas day
 i just want you here tonight
 holding on to me so tight
 girl what can i do
 you know that all i want for christmas
 is you
 and all the lights are shining
 so brightly everywhere
 and the sound of children's
 laughter fills the air
 and everyone is singing
 i can hear those sleigh bells ringing
 santa won't you bring me the one i really love
 won't you please bring my bab